<a href="https://colab.research.google.com/github/HarolReyes0/VRP-solved-by-ACO/blob/main/VRP_With_ACO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Solving the **Vehicle Routing Problem** using Ant **Colony Optimization Algorithm**.

Harol Reyes

2023-0380



Ex data iput:

* A B 5
* A C 8
* C B 7




In [1]:
import numpy as np

##Constructing the graphs.

In [2]:
class Graph:

  def __init__(self):
    self.distance_matrix = None
    self.nodes = {}

  def _init_distance_matrix(self, n):
    """
      Initialization of the distance matrix.

      Input:
            n: Number of nodes.
      Output:
            A matrix n x n full of infinite numbers.
    """
    self.distance_matrix = np.full((n,n), np.inf)

  def _init_pheromone_matrix(self, n):
    """
      Initialization of the distance matrix.

      Input:
            n: size of the matrix.
      Output:
            A matrix n x n full of ones.
    """
    self.pheromone_matrix = np.ones(n)

  def add_node(self, node_name):
    """
      Create a dictionary with the node name and a positional value.

      Inputs:
            node_name: The name of all the nodes.
      Output:
            A dictionary with the node name and a positional value

    """
    self.nodes[node_name] = len(self.nodes)

  def add_edge(self, node_a, node_b, dist):
    """
      Adds the edges and edges values to the matrix.
      inputs: node_a: a node.
              node_b: the node that wants to be connected.
              dist: the distance between nodes.

      Output: A matrix filled up with the edges values (distance).
    """
    self.distance_matrix[self.nodes[node_a]][self.nodes[node_b]] = dist
    self.distance_matrix[self.nodes[node_b]][self.nodes[node_a]] = dist

  def transform_txt(self, path):
    """
      Open and read the txt file, extract all the nodes names and makes a list
      with the edges.

      Inputs:
            Path: Where the txt file is store at.
      Output:
            nodes_names: A dictionary of all nodes found in the file)
            raw_nodes: edge of the nodess and its weight)
    """

    self.text = open(path)
    self.text = self.text.read()

    #Deliting the jumps
    self.nodes_names = self.text.replace("\n", "")
    #Getting the unique values and sorting them
    self.nodes_names = list(set([digit for digit in self.nodes_names
                          if not digit.isdigit() and digit.isalpha()]))
    self.nodes_names.sort()

    #Formating the raw data to create the following format [Node_a, Node_b and distance]
    self.edges = self.text.split("\n")

    return self.nodes_names, self.edges

  def graph_constructer(self, path):
    """
      This function inplement all the constructors to complete the process of
      getting the data and transform it in to a distance matrix.
    """
    self.nodes_names, self.edges = self.transform_txt(path)
    self._init_distance_matrix(len(self.nodes_names))

    for node_name in self.nodes_names:
      self.add_node(node_name)
    for edge in self.edges:
      #Takes values from position 0,2,3 (Node_a, Node_b and distance) and adds the edges
      self.add_edge(edge[0],edge[2],edge[4])
    self._init_pheromone_matrix(self.distance_matrix.shape)


##Making movements.

In [3]:
def pick_node(position, distance, pheromones, alpha = 1, beta = 1):
  """
    This funtion takes the node state of the ant and calculate the probabilities
    of choosing all the nodes and pick one node.

    inputs:
          position: the node where the ant is at.
          distance: the distance matrix.
          alpha: influence of the pheromones.
          beta: influence of the heuristic.

    output:
          The nex node that will be visited.
  """
  #calculates the distance matrix's inverse
  distance = 1/distance

  #calculates Σt^α * n^β
  denominator = [pheromones[position][node] ** alpha *
                 distance[position][node] ** beta
                for node in range(len(distance[position]))]

  #calculates Σt^α * n^β/Σt^α * n^β
  probabilities = [numerator/sum(denominator) for numerator in denominator]

  #Takes the probabilities and choose on node to move.
  return np.random.choice(len(probabilities) , p = probabilities)

In [4]:
def all_explored(n_ants, paths):
  """
      Checks if all nodes are explored.

      inputs:
            n_ants: number of ants.
            paths: paths created by the ants.
      outputs:
              Boolean.
  """
  n_nodes = set()

  #Filtering all the visited nodes.
  for ant in range(n_ants):
    for node in paths[ant]:
      n_nodes.add(node)

  #Returning True or False
  return len(g.nodes.keys()) == len(n_nodes)

In [5]:
def deleting_visited_rows(pheromones,position):
  """
      Deletes the nodes visited.

      inputs:
            pheromones: pheromone matrix.
            position: the ant's position
  """

  #Recorring the pheromone matrix and adding 0 in all the colums which has
  #been visited.
  for row in range(len(g.distance_matrix)):
        pheromones[row][position] = 0

##Pheromone actualization.

In [6]:
def pheromone_deposit(paths_costs, pheromone_n):
  """
    Calulates the pheromone deposit.

    Inputs:
          path_costs: Costs of all the pahts found.
          pheromone_n: base number of pheromones that wants to be depossited.

    Ouput:
          The base pheromones divided by the min path cost found.
  """
  return pheromone_n/sum(paths_costs.values())


In [7]:
def pheromone_evaporation(paths, n_pheromones, paths_costs, evaporation_rate):

  """
    This function takes the pheromones after the evaporation rate and puts them
    in the arc.

    input:
          paths: paths travelled.
          n_pheromones: pheromones that wants to be deposited.
          path_cost: the distance traveled by the ants.
          evaporation_rate: evaporation rate of the pheromones

    Outputs:
            Pheromone matrix updated with the correct pheromone values.
  """
  #Calculing (1-p) * t
  g.pheromone_matrix = (1 - evaporation_rate) * g.pheromone_matrix

  #fingind the pheromone deposit.
  deposit = pheromone_deposit(paths_costs, n_pheromones)

  #Getting the paht tha will work on
  for path in range(len(paths)):
    for node in range(len(paths[path])-1):
      g.pheromone_matrix[paths[path][node]][paths[path][node+1]] = deposit

##Solution development.

In [22]:
def path_construction(n_iteration, n_ants,n_pheromones, evaporation_rate, alpha = 1, beta = 1):
  """
    This funtion creates the path constructed by the ants.

    Input:
          n_ants: number of ants.
          alpha: Influence of the pheromones in the path construction.
          beta: Influence of the heuristic in the path construction.

    Outputs:
            paths: Paths constructed by the ants.
            path_cost: the cost of every path constructed by the ants.
  """
  for i in range(n_iteration):
    print("\n", "Iteracion numero: {}".format(i+1) , "\n")

    #Initialazing the dictionaries where the path and path cost is going to be saved.
    paths = {ant: [ant * 0] for ant in range(n_ants)}
    path_cost = {ant: ant * 0 for ant in range(n_ants)}
    distance = g.distance_matrix.copy()
    pheromones = g.pheromone_matrix.copy()

    while True:
      #Iterating throught the amount of ants that we have.
      for ant in range(n_ants):
        if path_cost[ant] != min(path_cost.values()):
          pass

        else:
          print(paths, "\n", path_cost, "\n")
          #Finding the last elment visited by the ant.
          position = paths[ant][-1]

          #The ant choose the next node that it'll visit.
          if not all_explored(n_ants, paths):
            new_position = int(pick_node(position, distance, pheromones, alpha, beta))
            deleting_visited_rows(pheromones,position)

          #updating path cost's value and the path found.
          path_cost[ant] += distance[position][new_position]
          paths[ant].append(int(new_position))

          #Deliting the nodes visited
          deleting_visited_rows(pheromones,new_position)

      #checking if all the nodes are explored
      if all_explored(n_ants, paths):
        path_cost[ant] += distance[position][new_position]
        paths[ant].append(int(new_position))
        break

    #calculating the pheromes that will be deposited
    pheromone_evaporation(paths, n_pheromones, path_cost, evaporation_rate)



##Results.

In [25]:
g = Graph()
g.graph_constructer("/content/Grafo 1.txt")
path_construction(20,2,5,0.5)


 Iteracion numero: 1 

{0: [0], 1: [0]} 
 {0: 0, 1: 0} 

{0: [0, 1], 1: [0]} 
 {0: 1.0, 1: 0} 

{0: [0, 1], 1: [0, 3]} 
 {0: 1.0, 1: 3.0} 

{0: [0, 1, 4], 1: [0, 3]} 
 {0: 9.0, 1: 3.0} 

{0: [0, 1, 4], 1: [0, 3, 2]} 
 {0: 9.0, 1: 4.0} 


 Iteracion numero: 2 

{0: [0], 1: [0]} 
 {0: 0, 1: 0} 

{0: [0, 1], 1: [0]} 
 {0: 1.0, 1: 0} 

{0: [0, 1], 1: [0, 2]} 
 {0: 1.0, 1: 2.0} 

{0: [0, 1, 3], 1: [0, 2]} 
 {0: 8.0, 1: 2.0} 

{0: [0, 1, 3], 1: [0, 2, 4]} 
 {0: 8.0, 1: 4.0} 


 Iteracion numero: 3 

{0: [0], 1: [0]} 
 {0: 0, 1: 0} 

{0: [0, 2], 1: [0]} 
 {0: 2.0, 1: 0} 

{0: [0, 2], 1: [0, 1]} 
 {0: 2.0, 1: 1.0} 

{0: [0, 2], 1: [0, 1, 3]} 
 {0: 2.0, 1: 8.0} 

{0: [0, 2, 4], 1: [0, 1, 3]} 
 {0: 4.0, 1: 8.0} 

{0: [0, 2, 4, 5], 1: [0, 1, 3]} 
 {0: 10.0, 1: 8.0} 


 Iteracion numero: 4 

{0: [0], 1: [0]} 
 {0: 0, 1: 0} 

{0: [0, 2], 1: [0]} 
 {0: 2.0, 1: 0} 

{0: [0, 2], 1: [0, 1]} 
 {0: 2.0, 1: 1.0} 

{0: [0, 2], 1: [0, 1, 3]} 
 {0: 2.0, 1: 8.0} 

{0: [0, 2, 4], 1: [0, 1, 3]} 
 {0: 4.0, 1: 8